In [25]:
from datasets import load_dataset
# DATASET_NAME='arc_easy'
# dataset = load_dataset("allenai/ai2_arc", "ARC-Easy", split=["train", "test"])
DATASET_NAME='arc_hard'
dataset = load_dataset("allenai/ai2_arc", "ARC-Challenge", split=["train", "test"])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1119 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1172 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/299 [00:00<?, ? examples/s]

In [26]:
import itertools

perm_order = list(itertools.permutations([0, 1, 2, 3], 4))

In [27]:
def get_prompt(dataset, index, has_choice=False):
    try:
        json_line = dataset[index]
        question = json_line["question"]
        choices = json_line["choices"]
        choice_texts = choices["text"]
        if len(choice_texts)<4:
            print(len(choice_texts))
            choice_texts.append(' ')
        candidates = " ".join(
            [
                f"({label if has_choice else ' '}) {text}"
                for text, label in zip(choice_texts, choices["label"])
            ]
        ).replace("\n", " ")
        answer_key = json_line["answerKey"]
        if answer_key[0] in '1234':
            answer_key_idx = ord(answer_key[0]) - ord("1")
        else:
            answer_key_idx = ord(answer_key[0]) - ord("A")
        answer_text = choices["text"][answer_key_idx]
        
        question_pad = f"{question} \\n "
        prompt = f"{question_pad}{candidates}"
        context=''
    except:
        print(question,choice_texts)
    return context,prompt, answer_text

In [28]:
from tqdm import tqdm
import pickle

for index, dataname in enumerate(['train','test']):
    container=[]
    for ques_index, ques in enumerate(tqdm(dataset[index])):
        container.append(
                get_prompt(dataset[index], ques_index, has_choice=False)
            )
    pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

100%|██████████| 1119/1119 [00:00<00:00, 4337.13it/s]


3


  0%|          | 0/1172 [00:00<?, ?it/s]

3


100%|██████████| 1172/1172 [00:00<00:00, 5215.47it/s]

3
3
3
